## ML model training 

In [6]:
# Import necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

In [7]:
# Load your symptom data
df = pd.read_csv("../data/sample_symptoms.csv", parse_dates=["date"])


In [11]:
# Drop any rows where flare is missing
df = df.dropna(subset=["flare"])

# Features and target
X = df[["fatigue", "pain", "brain_fog"]]


In [12]:
# Map categorical values in the 'flare' column to integers
flare_mapping = {"low risk": 0, "medium risk": 1, "high risk": 2}  # Adjust mapping as needed
df["flare"] = df["flare"].map(flare_mapping)

# Drop rows with unmapped values (if any)
df = df.dropna(subset=["flare"])

# Convert to integer
y = df["flare"].astype(int)

In [13]:
# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Train a Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save the trained model
joblib.dump(model, "../backend/flare_model.pkl")
print("Model saved to backend/flare_model.pkl")